In [1]:
import tensorflow as tf
from music21 import converter, instrument, stream, roman, midi, key, interval, duration
import music21
import music21.chord as chord_module
import music21.note as note_module
import glob
import numpy as np
import json
import pickle
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, LSTM, Activation, Embedding, Concatenate, Input, Bidirectional, Attention
from keras.layers import BatchNormalization as BatchNorm
from tensorflow.keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import EarlyStopping
import re
from pathlib import Path
import ast


2024-05-08 21:40:32.288322: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-08 21:40:32.339310: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-08 21:40:32.626569: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/jupyter-franky/.conda/envs/tf/lib/
2024-05-08 21:40:32.626600: W tensorflow/compiler/xla/st

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


2024-05-08 12:19:50.680140: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-05-08 12:19:50.682838: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-05-08 12:19:50.682897: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [6]:
def get_midi_files_pathlib(directory):
    path = Path(directory)
    files_lower = list(path.glob('*.mid'))
    files_upper = list(path.glob('*.MID'))

    all_files = files_lower + files_upper
    return all_files

directory = 'dataset/choral-dataset-2/'
midi_files = get_midi_files_pathlib(directory)

In [7]:
duration_mapping = {
    0.25: 0,  # Sixteenth note
    0.5: 1,   # Eighth note
    0.75: 2,
    1.0: 3,   # Quarter note
    1.25: 4,
    1.5: 5,   # Dotted quarter note
    1.75: 6,
    2.0: 7,   # Half note
    2.25: 8,
    2.5: 9,   
    2.75: 10,
    3.0: 11,   
    3.25: 12,  
    3.5: 13,   
    3.75: 14, 
    4.0: 15    # Whole note
}

reverse_duration_mapping = {value: key for key, value in duration_mapping.items()}

In [8]:
def get_duration(duration, duration_mapping=duration_mapping):
    min_diff = float('inf')
    nearest_duration = None
    for key in duration_mapping:
        diff = abs(duration - key)
        if diff < min_diff:
            min_diff = diff
            nearest_duration = key
    return duration_mapping[nearest_duration]
    
def extract_midi_file(midi_file):
    midi = converter.parse(midi_file)
    original_key = midi.analyze('key')
    if str(original_key) != "C major":      
        transposed_key = key.Key('C')
        interval = music21.interval.Interval(original_key.tonic, transposed_key.tonic)
        midi = midi.transpose(interval)
    else:
        transposed_key = original_key
            
    melody_stream = stream.Score()
    chord_stream = stream.Score()
    
    melody_part = midi.parts[0]
    melody_stream.append(melody_part)
    
    if not melody_stream.hasMeasures():
        melody_stream.makeMeasures(inPlace=True)
        
    for part in midi.parts[1:]:
        chord_stream.append(part)
    chords = chord_stream.chordify()

    melody_measures = list(melody_part.measures(0, None))
    chord_measures = list(chords.measures(0, None))
    
    melody_notes = []
    melody_duration = []
    chord_notes = []
    chord_duration = []
    

    for melody_measure, chord_measure in zip(melody_measures, chord_measures):
        melody, mel_duration = extract_measure_info(melody_measure)
        chord, ch_duration  = extract_measure_info(chord_measure)
        
        melody_notes.append(melody)
        melody_duration.append(mel_duration)
        chord_notes.append(chord)
        chord_duration.append(ch_duration)
        
    melody_notes.append(['<end>'])
    melody_duration.append([0])  # Duration for end can be zero or another suitable value
    chord_notes.append(['<end>'])
    chord_duration.append([0])
    
    return melody_notes, melody_duration, chord_notes, chord_duration
    
def extract_measure_info(measure):
    """ Extracts notes and mapped durations from a given measure """
    notes = []
    durations = []
    for element in measure.notesAndRests:
        if isinstance(element, note_module.Note):
            notes.append(element.pitch.midi)
            mapped_duration = get_duration(float(element.duration.quarterLength), duration_mapping)
            durations.append(mapped_duration)
        elif isinstance(element, chord_module.Chord):
            # For chords, consider each note's pitch in the chord
            chord_notes = [p.midi for p in element.pitches]
            notes.append(chord_notes)
            mapped_duration = get_duration(float(element.duration.quarterLength), duration_mapping)
            durations.append(mapped_duration)
        elif isinstance(element, note_module.Rest):
            notes.append('Rest')
            mapped_duration = get_duration(float(element.duration.quarterLength), duration_mapping)
            durations.append(mapped_duration)
    return notes, durations



In [9]:
print(len(midi_files))
all_files_notes = []
all_files_durations = []
all_files_chords = []
all_files_chord_durations = []
count = 0
for file in midi_files:
    notes, durations, chords, chord_durations = extract_midi_file(file)
    if notes is not None and durations is not None and chords is not None and chord_durations is not None:
        all_files_notes.extend(notes)
        all_files_durations.extend(durations)
        all_files_chords.extend(chords)
        all_files_chord_durations.extend(chord_durations)
        count += 1
        print(count)
    else:
        print(f"Skipping {file} due to errors in extraction.")

182
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56


/home/jupyter-franky/.conda/envs/tf/lib/python3.9/site-packages/music21/midi/translate.py:863: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=0, channel=None, data=b'\xfe\xff\x00L\x00a\x00m\x00e\x00n\x00t\x00a\x00t\x00i\x00o\x00 \x00J\x00e\x00r\x00e\x00m\x00i\x00a\x00e\x00 \x00\xe0\x00 \x003\x00 \x00(\x00S\x00T\x00B\x00)'>; getting generic Instrument
  warnings.warn(


57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182


In [15]:
print(len(all_files_notes), len(all_files_durations), len(all_files_chords), len(all_files_chord_durations))

12180 12180 12180 12180


In [16]:
print(all_files_notes[:10])
print(all_files_durations[:10])
print(all_files_chords[:10])
print(all_files_chord_durations)

[['Rest', 63, 62, 63], [65, 63, 62, 63], [60, 58, 60], [58], ['Rest', 63, 62, 63], [65, 63, 62, 63], [60, 58, 60], [58], ['Rest', 67, 67, 67], [67, 67, 67, 67]]
[[3, 3, 3, 3], [5, 1, 3, 3], [3, 3, 7], [7], [3, 3, 3, 3], [5, 1, 3, 3], [3, 3, 7], [7], [3, 3, 3, 3], [5, 1, 3, 3]]
[['Rest', [48, 60], [43, 58], [48, 60]], [[46, 62], [48, 60], [43, 58], [39, 55]], [[41, 57], [43, 58], [41, 58], [41, 57]], [[34, 58]], ['Rest', [48, 60], [43, 58], [48, 60]], [[46, 62], [48, 60], [43, 58], [39, 55]], [[41, 57], [43, 58], [41, 58], [41, 57]], [[34, 58]], ['Rest', [39, 63], [39, 63], [39, 63]], [[39, 63], [39, 63], [39, 63], [39, 63]]]
[[3, 3, 3, 3], [5, 1, 3, 3], [3, 3, 3, 3], [7], [3, 3, 3, 3], [5, 1, 3, 3], [3, 3, 3, 3], [7], [3, 3, 3, 3], [5, 1, 3, 3], [3, 3, 7], [3, 3, 0, 0, 0, 0, 1, 1], [3, 1, 1, 3, 3], [3, 3, 3, 3], [3, 3, 7], [3, 3, 3, 3], [3, 3, 5, 1], [3, 3, 7], [3, 7, 3], [3, 3, 3, 3], [15], [3, 3, 3, 3], [5, 1, 3, 3], [3, 3, 7], [3, 3, 0, 0, 0, 0, 1, 1], [3, 1, 1, 3, 3], [3, 3, 3, 3],

In [17]:
def create_mappings(items_list, mapping_path):
    unique_items = set()  # Use a set to automatically handle unique entries
    for items in items_list:
        item_str = '_'.join(str(i) for i in items)
        unique_items.add(item_str)  
    
    unique_items = list(unique_items)
    
    mappings = {item: number for number, item in enumerate(unique_items)}
    with open(mapping_path, "w") as file:
        json.dump(mappings, file)
    return mappings

def convert_to_int(items, mapping):
    int_notes = []
    
    for item in items:
        if isinstance(item, list):
            item_str = '_'.join(str(i) for i in item)
        else:
            item_str = item
        
        int_notes.append(mapping.get(item_str, -1))
    return int_notes

In [18]:
note_mapping_path = "mappings_model/note_mappings.json"
note_duration_mapping_path = "mappings_model/note_duration_mappings.json"
chord_mapping_path = "mappings_model/chord_mappings.json"
chord_duration_mapping_path = "mappings_model/chord_duration_mappings.json"

notes_mapping = create_mappings(all_files_notes, note_mapping_path)
durations_mapping = create_mappings(all_files_durations, note_duration_mapping_path)
chords_mapping = create_mappings(all_files_chords, chord_mapping_path)
chord_durations_mapping = create_mappings(all_files_chord_durations, chord_duration_mapping_path)

In [19]:
print(notes_mapping)
print(durations_mapping)
print(chords_mapping)
print(chord_durations_mapping)

{'65_Rest_69_Rest_67': 0, '67_65_63_65_67_63': 1, '74_74_69_71_72_72_71': 2, '60_67_69_67': 3, '72_69_67': 4, '41_53': 5, '71_72_71_69_71': 6, 'Rest_67_Rest_65_Rest_63_Rest': 7, '60_60_58_58': 8, '60_59_60_60_60': 9, '[60, 57]_[59, 55]_[57, 53]_[55, 52]': 10, '62_63_65_62_63_65': 11, '62_62_58': 12, '67_Rest_60_72': 13, '69_Rest_69': 14, '63_65_67_Rest_67': 15, '69_65_65_67': 16, '62_63_65_62_63_65_67': 17, '51_Rest_50': 18, '64_62_60_67': 19, '69_71_71': 20, '62_63_65_63_62_62': 21, '71_69_65': 22, '55_51': 23, 'Rest_70_70': 24, '72_72_69_69_70': 25, '70_69_67_65_63_62_63': 26, '64_62_60_62_64_65_67_65_64': 27, '64_65_67_64_66_67_66_67': 28, '67_69_70': 29, '65_65_64_64_62': 30, '68_65_67': 31, '72_Rest_67': 32, '65_63_62_60': 33, '69_72_Rest_70_Rest': 34, '50_48_46_45': 35, '[48, 40]_40_38_[47, 40]_[45, 41]': 36, '65_67_69_67': 37, '55_67_67_67_65_63_62': 38, '53_53_57_57': 39, '60_58_60_Rest': 40, '67_62_64': 41, '65_67_65_64': 42, '67_65_64_65_64_64_62_60_59': 43, 'Rest_64_62_60_60

In [20]:
def preprocess_sequences(notes, note_durations, chords, chord_durations, sequence_length, notes_mapping, chords_mapping, note_duration_mapping, chord_duration_mapping):
    
    encoded_notes = convert_to_int(notes, notes_mapping)
    encoded_note_durations = convert_to_int(note_durations, note_duration_mapping)
    encoded_chords = convert_to_int(chords, chords_mapping)
    encoded_chord_durations = convert_to_int(chord_durations, chord_duration_mapping)
    
    input_notes, input_note_durations, input_chords, input_chord_durations, output_notes, output_note_durations, output_chords, output_chord_durations = [], [], [], [], [], [], [], []
    
    for i in range(len(encoded_notes) - sequence_length):
        input_notes.append(encoded_notes[i:i+sequence_length])
        input_note_durations.append(encoded_note_durations[i:i+sequence_length])
        input_chords.append(encoded_chords[i:i+sequence_length])
        input_chord_durations.append(encoded_chord_durations[i:i+sequence_length])
        
        output_notes.append(encoded_notes[i+sequence_length])
        output_note_durations.append(encoded_note_durations[i+sequence_length])
        output_chords.append(encoded_chords[i+sequence_length])
        output_chord_durations.append(encoded_chord_durations[i+sequence_length])
        
    n_vocab_notes = len(notes_mapping)
    n_vocab_durations = len(note_duration_mapping)
    n_vocab_chords = len(chords_mapping)
    n_vocab_chord_durations = len(chord_duration_mapping)

    return np.array(input_notes), np.array(input_note_durations), np.array(input_chords), np.array(input_chord_durations), np.array(output_notes), np.array(output_note_durations), np.array(output_chords), np.array(output_chord_durations), n_vocab_notes, n_vocab_durations, n_vocab_chords, n_vocab_chord_durations

In [21]:
sequence_length = 64
input_notes, input_note_durations, input_chords, input_chord_durations, output_notes, output_note_durations, output_chords, output_chord_durations, n_vocab_notes, n_vocab_durations, n_vocab_chords, n_vocab_chord_durations = preprocess_sequences(all_files_notes, all_files_durations, all_files_chords, all_files_chord_durations, sequence_length, notes_mapping, chords_mapping, durations_mapping, chord_durations_mapping)

In [22]:
print(n_vocab_notes, n_vocab_durations, n_vocab_chords, n_vocab_chord_durations)

3696 757 8232 1088


In [23]:
print(input_notes[:5])
print(input_note_durations[:5])
print(input_chords[:5])
print(input_chord_durations[:5])

[[2482 2502   62 1728 2482 2502   62 1728  398  589 2896 1057 3601 3247
  2593  398 2093 2445 2749  372 2264  398  589 2896 1057 3601 3247 2593
   398 2093 2445 2749  372 2264 1027 3567 3567 1646  955 2705  117  653
  1275 2782 1483 2006  500 1980  988 3567 3567 3567 3567 3567   66 2561
  2755 2648 3479 3438 1885  762 3567 1885]
 [2502   62 1728 2482 2502   62 1728  398  589 2896 1057 3601 3247 2593
   398 2093 2445 2749  372 2264  398  589 2896 1057 3601 3247 2593  398
  2093 2445 2749  372 2264 1027 3567 3567 1646  955 2705  117  653 1275
  2782 1483 2006  500 1980  988 3567 3567 3567 3567 3567   66 2561 2755
  2648 3479 3438 1885  762 3567 1885  117]
 [  62 1728 2482 2502   62 1728  398  589 2896 1057 3601 3247 2593  398
  2093 2445 2749  372 2264  398  589 2896 1057 3601 3247 2593  398 2093
  2445 2749  372 2264 1027 3567 3567 1646  955 2705  117  653 1275 2782
  1483 2006  500 1980  988 3567 3567 3567 3567 3567   66 2561 2755 2648
  3479 3438 1885  762 3567 1885  117 1557]
 [1728 

In [24]:
print(len(input_notes), len(input_note_durations), len(input_chords), len(input_chord_durations), len(output_notes), len(output_note_durations), len(output_chords), len(output_chord_durations), n_vocab_notes, n_vocab_durations, n_vocab_chords, n_vocab_chord_durations)

12116 12116 12116 12116 12116 12116 12116 12116 3696 757 8232 1088


In [25]:
def hierarchical_lstm_model(sequence_length, n_vocab_notes, n_vocab_chords, n_vocab_melody_durations, n_vocab_chord_durations, note_embedding_dim, chord_embedding_dim, melody_duration_embedding_dim, chord_duration_embedding_dim, lstm_units, dropout_rate, learning_rate):
    # Note input and embedding
    note_input = Input(shape=(sequence_length,), name='note_input')
    note_embedding = Embedding(input_dim=n_vocab_notes + 1, output_dim=note_embedding_dim, input_length=sequence_length, name='note_embedding')(note_input)

    # Melody duration input and embedding
    melody_duration_input = Input(shape=(sequence_length,), name='melody_duration_input')
    melody_duration_embedding = Embedding(input_dim=n_vocab_melody_durations + 1, output_dim=melody_duration_embedding_dim, input_length=sequence_length, name='melody_duration_embedding')(melody_duration_input)

    # Chord input and embedding
    chord_input = Input(shape=(sequence_length,), name='chord_input')
    chord_embedding = Embedding(input_dim=n_vocab_chords + 1, output_dim=chord_embedding_dim, input_length=sequence_length, name='chord_embedding')(chord_input)

    # Chord duration input and embedding
    chord_duration_input = Input(shape=(sequence_length,), name='chord_duration_input')
    chord_duration_embedding = Embedding(input_dim=n_vocab_chord_durations + 1, output_dim=chord_duration_embedding_dim, input_length=sequence_length, name='chord_duration_embedding')(chord_duration_input)

    # Concatenate embeddings
    note_melody_concat = Concatenate(name='note_melody_concat')([note_embedding, melody_duration_embedding])
    note_melody_lstm = LSTM(lstm_units, return_sequences=True, name='note_melody_lstm')(note_melody_concat)
    note_melody_dropout = Dropout(dropout_rate, name='note_melody_dropout')(note_melody_lstm)

    chord_chord_duration_concat = Concatenate(name='chord_duration_concat')([chord_embedding, chord_duration_embedding])
    chord_lstm = LSTM(lstm_units, return_sequences=True, name='chord_lstm')(chord_chord_duration_concat)
    chord_dropout = Dropout(dropout_rate, name='chord_dropout')(chord_lstm)

    # Combine LSTM outputs
    combined = Concatenate(name='concatenate')([note_melody_dropout, chord_dropout])
    combined_lstm = Bidirectional(LSTM(lstm_units, name='combined_lstm'))(combined)
    combined_dropout = Dropout(dropout_rate, name='combined_dropout')(combined_lstm)

    # Output layers
    note_output = Dense(n_vocab_notes + 1, activation='softmax', name='note_output')(combined_dropout)
    melody_duration_output = Dense(n_vocab_melody_durations + 1, activation='softmax', name='melody_duration_output')(combined_dropout)
    chord_output = Dense(n_vocab_chords + 1, activation='softmax', name='chord_output')(combined_dropout)
    chord_duration_output = Dense(n_vocab_chord_durations + 1, activation='softmax', name='chord_duration_output')(combined_dropout)

    # Model compilation
    model = Model(inputs=[note_input, melody_duration_input, chord_input, chord_duration_input], outputs=[note_output, melody_duration_output, chord_output, chord_duration_output])
    model.compile(optimizer=Adam(learning_rate=learning_rate, clipnorm=1.0),
                  loss={'note_output': 'sparse_categorical_crossentropy',
                        'melody_duration_output': 'sparse_categorical_crossentropy',
                        'chord_output': 'sparse_categorical_crossentropy',
                        'chord_duration_output': 'sparse_categorical_crossentropy'},
                  loss_weights={'note_output': 1.0, 'melody_duration_output': 1.0, 'chord_output': 1.0, 'chord_duration_output': 1.0})

    return model

In [26]:
EPOCHS = 200
BATCH_SIZE = 128
learning_rate = 0.0005
sequence_length = 64
note_embedding_dim = 150
chord_embedding_dim = 300
melody_duration_embedding_dim = 50
chord_duration_embedding_dim = 50
lstm_units = 256
dropout_rate = 0.4
MODEL_PATH = "mappings_model/choral-model.h5"
output_path = "mappings_model/choral-output.mid"

In [41]:
def train(continue_training=False):
    network_input = [input_notes, input_note_durations, input_chords, input_chord_durations]
    network_output = [output_notes, output_note_durations, output_chords, output_chord_durations]
    
    if continue_training:
        # Load the previously trained model
        model = load_model(MODEL_PATH)
    else:
        # Create a new model
        model = hierarchical_lstm_model(sequence_length, n_vocab_notes, n_vocab_chords, 
                                        n_vocab_durations, n_vocab_chord_durations, 
                                        note_embedding_dim, chord_embedding_dim, 
                                        melody_duration_embedding_dim, chord_duration_embedding_dim, 
                                        lstm_units, dropout_rate, learning_rate)
    
    early_stopping = EarlyStopping(
        monitor='loss',  # Monitor the training loss
        patience=10,  # Number of epochs to wait for improvement
        verbose=1,  # Print messages when early stopping is triggered
        mode='min'  # Look for a minimum value of the monitored metric
    )

    # Training the model
    model.fit(network_input, network_output,
              epochs=EPOCHS,
              batch_size=BATCH_SIZE,
              callbacks=[early_stopping])
    
    # Save the trained model
    model.save(MODEL_PATH)


train(True)

Epoch 1/200
95/95 [==============================] - 3s 16ms/step - loss: 0.2009 - note_output_loss: 0.0528 - melody_duration_output_loss: 0.0619 - chord_output_loss: 0.0165 - chord_duration_output_loss: 0.0697
Epoch 2/200
95/95 [==============================] - 2s 16ms/step - loss: 0.2021 - note_output_loss: 0.0505 - melody_duration_output_loss: 0.0602 - chord_output_loss: 0.0146 - chord_duration_output_loss: 0.0767
Epoch 3/200
95/95 [==============================] - 2s 16ms/step - loss: 0.1971 - note_output_loss: 0.0490 - melody_duration_output_loss: 0.0585 - chord_output_loss: 0.0161 - chord_duration_output_loss: 0.0736
Epoch 4/200
95/95 [==============================] - 2s 16ms/step - loss: 0.2073 - note_output_loss: 0.0517 - melody_duration_output_loss: 0.0645 - chord_output_loss: 0.0157 - chord_duration_output_loss: 0.0753
Epoch 5/200
95/95 [==============================] - 2s 16ms/step - loss: 0.2115 - note_output_loss: 0.0551 - melody_duration_output_loss: 0.0629 - chord_ou

In [49]:
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)

def temperature_sampling(predictions, temperature=1.0):
    scaled = np.log(predictions + 1e-7) / temperature  
    probabilities = softmax(scaled)
    choice = np.random.choice(len(probabilities), p=probabilities)
    return choice
    
def predict_notes(model_path, starting_notes, starting_melody_durations, starting_chords, starting_chord_durations, sequence_length, n_notes=300, temperature=1.0):
    model = load_model(model_path)

    prediction_output_notes = []
    prediction_output_melody_durations = []
    prediction_output_chords = []
    prediction_output_chord_durations = []
    
    with open(note_mapping_path, "r") as f:
        note_mappings = json.load(f)
    with open(note_duration_mapping_path, "r") as f:
        melody_duration_mappings = json.load(f)
    with open(chord_duration_mapping_path, "r") as f:
        chord_duration_mappings = json.load(f)
    with open(chord_mapping_path, "r") as f:
        chord_mappings = json.load(f)

    reverse_note_mappings = {value: key for key, value in note_mappings.items()}
    reverse_melody_duration_mappings = {value: key for key, value in melody_duration_mappings.items()}
    reverse_chord_duration_mappings = {value: key for key, value in chord_duration_mappings.items()}
    reverse_chord_mappings = {value: key for key, value in chord_mappings.items()}

    # Convert starting inputs to their encoded forms
    starting_notes_int = convert_to_int(starting_notes, note_mappings)
    starting_melody_durations_int = convert_to_int(starting_melody_durations, melody_duration_mappings)
    starting_chords_int = convert_to_int(starting_chords, chord_mappings)
    starting_chord_durations_int = convert_to_int(starting_chord_durations, chord_duration_mappings)


    for _ in range(n_notes):
        # Prepare the input for prediction
        input_sequence_notes = np.array([starting_notes_int[-sequence_length:]])
        input_sequence_melody_durations = np.array([starting_melody_durations_int[-sequence_length:]])
        input_sequence_chords = np.array([starting_chords_int[-sequence_length:]])
        input_sequence_chord_durations = np.array([starting_chord_durations_int[-sequence_length:]])

        # Model prediction
        prediction = model.predict([input_sequence_notes, input_sequence_melody_durations, input_sequence_chords, input_sequence_chord_durations], verbose=0)
        next_note = temperature_sampling(prediction[0][0], temperature)
        next_melody_duration = temperature_sampling(prediction[1][0], temperature)
        next_chord = temperature_sampling(prediction[2][0], temperature)
        next_chord_duration = temperature_sampling(prediction[3][0], temperature)

        # Update the sequences with the predicted values
        starting_notes_int.append(next_note)
        starting_melody_durations_int.append(next_melody_duration)
        starting_chords_int.append(next_chord)
        starting_chord_durations_int.append(next_chord_duration)

        # Convert predictions back to original values and store
        prediction_output_notes.append(reverse_note_mappings.get(next_note, "Unknown"))
        prediction_output_melody_durations.append(reverse_melody_duration_mappings.get(next_melody_duration, "Unknown"))
        prediction_output_chords.append(reverse_chord_mappings.get(next_chord, "Unknown"))
        prediction_output_chord_durations.append(reverse_chord_duration_mappings.get(next_chord_duration, "Unknown"))

    return prediction_output_notes, prediction_output_melody_durations, prediction_output_chords, prediction_output_chord_durations



In [56]:
seed = 100
starting_notes = all_files_notes[seed:seed+sequence_length]
starting_melody_durations = all_files_durations[seed:seed+sequence_length]
starting_chords = all_files_chords[seed:seed+sequence_length]
starting_chord_durations = all_files_chord_durations[seed:seed+sequence_length]

In [57]:
prediction_notes, prediction_melody_durations, prediction_chords, prediction_chord_durations = predict_notes(MODEL_PATH, starting_notes, starting_melody_durations, starting_chords, starting_chord_durations, sequence_length, n_notes=200, temperature=1.0)

In [58]:
print(starting_notes)

[[63], [65, 67], [65, 63], [62, 62, 65], [65, 67], [69, 70], [69], [67, 'Rest', 67], [67, 70, 69, 69], [67, 65], ['Rest', 70], [67, 65, 63, 62], [60, 60, 63], [62, 60, 62], [60], [63], [65, 67], [65, 63], [62, 62, 65], [65, 67], [69, 70], [69], [67, 'Rest', 67], [67, 70, 69, 69], [67, 65], ['Rest', 70], [67, 65, 63, 62], [60, 60, 63], [62, 60, 62], [60], ['<end>'], ['Rest'], ['Rest'], [60, 63, 63], [62, 67, 60, 63, 62], [60, 'Rest', 62, 62], [63, 67, 63, 62, 60, 59, 60, 59], [60, 64], [65, 65, 63, 63], [62, 63, 65, 67], ['Rest', 60, 60], [57, 53, 60, 58, 60, 62, 63, 62, 63, 65, 67], [67, 66, 67], ['Rest'], ['Rest'], ['Rest'], ['Rest', 62, 62, 65, 63], [62, 60, 58, 58], [57, 60], [62, 62, 63, 67, 67], [65, 63, 62, 60, 60], [60, 65, 65, 65, 65, 63], [63, 62, 62, 62], ['Rest', 55, 57, 58, 60, 62, 63, 65, 63], [62, 60], ['Rest'], ['Rest', 60], [62, 63, 65, 65, 65, 65], [65, 65, 65, 63, 63, 60], [60, 60, 60, 63, 65, 63], [62, 67], [67, 65, 63, 62, 63, 62, 60, 58], [58], ['Rest']]


In [59]:
# print(prediction_notes[30:100])
# print(prediction_melody_durations[30:100])
print(prediction_notes[33:35])
print(prediction_chord_durations[33:35])

['72_75_74_72_70', '69_67_69']
['3_3_3_1_1', '3_7_3']


In [60]:
def create_midi(predictions_notes, predictions_durations, predictions_chords, predictions_chord_durations, file_name="output.mid"):
    # Create music21 stream for melody and chords
    s = stream.Score()
    melody_part = stream.Part()
    chord_part = stream.Part()
    
    for i, (notes, dur, chords, chord_dur) in enumerate(zip(predictions_notes, predictions_durations, predictions_chords, predictions_chord_durations)):
        note_list = notes.split('_')
        duration_list = dur.split('_')
        chord_list = chords.split('_')
        chord_duration_list = chord_dur.split('_')

        if len(duration_list) < len(note_list):
            last_duration = duration_list[-1] if duration_list else 1
            duration_list.extend([last_duration] * (len(note_list) - len(duration_list)))

        if len(chord_duration_list) < len(chord_list):
            last_chord_duration = chord_duration_list[-1] if chord_duration_list else 1
            chord_duration_list.extend([last_chord_duration] * (len(chord_list) - len(chord_duration_list)))
            
        for j in range(len(note_list)):
            if note_list[j] == '<end>':
                break
            elif note_list[j] == 'Rest':
                n = note_module.Rest()
            else:
                 try:
                    note_value = ast.literal_eval(note_list[j])
                    if isinstance(note_value, list):
                        pitches = [int(p) for p in note_value]
                        n = chord_module.Chord(pitches)
                    else:
                        n = note_module.Note()
                        n.pitch.midi = int(note_value)
                 except: 
                    n = note_module.Note()
                    n.pitch.midi = int(note_list[j])
            n.duration.quarterLength = float(reverse_duration_mapping.get(int(duration_list[j]), 1.0))
        
            melody_part.append(n)
            
        for k in range(len(chord_list)):
            chord_data = chord_list[k]
            if chord_list[k] == '<end>':
                break
            elif chord_list[k] == 'Rest':
                c = note_module.Rest()
            else:
                chord_pitches = chord_list[k].split('_')
                for chord_str in chord_pitches:
                    if chord_str == 'Rest':
                        c = note_module.Rest()
                    else:
                        chord_pitches = chord_str.strip('[]').split(',')
                        chord_pitches = [int(p) for p in chord_pitches]
                        c = chord_module.Chord(chord_pitches)
            print(float(reverse_duration_mapping.get(int(chord_duration_list[k]))))
            c.duration.quarterLength = float(reverse_duration_mapping.get(int(chord_duration_list[k]), 1.0))
            chord_part.append(c)
    s.insert(0, melody_part)
    s.insert(0, chord_part)
    s.write('midi', fp=file_name)   

In [61]:
create_midi(prediction_notes, prediction_melody_durations, prediction_chords, prediction_chord_durations, output_path)

2.0
1.0
1.0
2.0
2.0
2.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.5
0.5
1.5
0.5
1.0
1.0
2.0
2.0
1.0
1.0
1.0
1.0
2.0
1.0
1.0
1.5
0.5
2.0
2.0
1.0
1.0
2.0
1.0
1.0
1.5
0.5
2.0
2.0
2.0
1.0
1.0
4.0
2.0
4.0
2.0
2.0
1.0
1.0
3.0
1.0
2.0
2.0
2.0
1.0
1.0
1.0
1.0
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
1.0
1.0
2.0
4.0
1.0
1.0
1.0
2.0
1.0
2.0
2.0
1.0
1.0
1.0
1.0
1.0
2.0
1.0
2.0
2.0
1.0
1.0
1.0
1.0
1.0
2.0
1.0
2.0
2.0
1.0
1.0
1.0
1.0
2.0
2.0
1.0
1.0
1.0
1.0
2.0
1.0
1.0
2.0
2.0
1.0
1.0
1.0
1.0
2.0
2.0
1.0
1.0
1.0
1.0
2.0
2.0
2.0
2.0
1.0
1.0
1.0
1.0
2.0
2.0
1.0
0.5
0.5
1.0
1.0
2.0
1.0
1.0
2.0
2.0
1.0
0.5
0.5
1.0
1.0
2.0
2.0
1.0
0.5
0.5
1.0
1.0
2.0
2.0
1.0
1.0
1.0
1.0
2.0
1.0
1.0
2.0
1.0
1.0
4.0
2.0
1.0
1.0
1.0
0.5
0.5
1.5
0.5
1.0
1.0
1.0
0.5
0.5
1.0
2.0
1.0
4.0
2.0
1.0
1.0
1.0
0.5
0.5
1.5
0.5
1.0
1.0
1.0
0.5
0.5
1.0
2.0
1.0
1.0
1.5
0.5
1.0
1.0
1.0
1.5
0.5
1.0
1.0
1.0
0.5
0.5
1.0
1.0
1.0
1.0
4.0
1.0
2.0
1.0
2.0
1.0
1.0
2.0
1.0
1.0
3.0
1.0
1.0
1.0
1.0
1.0
2.0
2.0
1.0
1.0
1.0
